In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
# mergedf = pd.read_csv('metadata_otu_5000_merged.csv', low_memory=False)
mergedf = pd.read_csv('DB_CSV_merged_v1.csv', low_memory=False)
mergedf = mergedf.drop("Unnamed: 0", axis=1)
mergedf.head(1)

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum,have_dog
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,no


In [ ]:
# # mergedf.body_site.value_counts()
# mergedf.shape
# col_list = mergedf.columns.tolist()
# col_list

In [3]:
# pd.melt(df, id_vars=['A'], value_vars=['B'],
#         var_name='myVarname', value_name='myValname')

meltdf = pd.melt(mergedf, id_vars=mergedf.iloc[:, 0:-1043], value_vars=mergedf.iloc[:, -1035:-1],
        var_name='Taxonomy', value_name='Reads')
print(meltdf.shape)
meltdf.head()

(1085700, 47)


,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,physical_specimen_location,physical_specimen_remaining,public,qiita_study_id,role,sample_type,scientific_name,sex,Taxonomy,Reads
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
1,797.02.HA.LeftPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
2,797.02.HA.RightPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
3,797.02.HA.Stool,1,1,years,Adolescent,FS_01,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,CCME,False,True,797,Offspring,stool,human gut metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
4,797.02.HA.Tongue,1,1,years,Adolescent,FS_01,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,CCME,False,True,797,Offspring,saliva,human oral metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0


In [4]:
# meltdf_nr = meltdf.replace(meltdf[>0], 1)  # df[df>9] = 1

# meltdf.loc[(meltdf['Reads'] > 0)] = 1
meltdf.loc[meltdf['Reads'] > 0, 'Reads'] = 1
meltdf

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,physical_specimen_location,physical_specimen_remaining,public,qiita_study_id,role,sample_type,scientific_name,sex,Taxonomy,Reads
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
1,797.02.HA.LeftPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
2,797.02.HA.RightPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Offspring,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
3,797.02.HA.Stool,1,1,years,Adolescent,FS_01,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,CCME,False,True,797,Offspring,stool,human gut metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
4,797.02.HA.Tongue,1,1,years,Adolescent,FS_01,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,CCME,False,True,797,Offspring,saliva,human oral metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
5,797.02.HB.Forehead,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,CCME,False,True,797,Partner,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
6,797.02.HB.LeftPalm,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Partner,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
7,797.02.HB.RightPalm,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,CCME,False,True,797,Partner,skin,human skin metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
8,797.02.HB.Stool,35,35,years,Adult,FS_02,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,CCME,False,True,797,Partner,stool,human gut metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0
9,797.02.HB.Tongue,35,35,years,Adult,FS_02,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,CCME,False,True,797,Partner,saliva,human oral metagenome,female,k__Bacteria; p__Bacteroidetes; c__Flavobacteri...,0


In [5]:
meltdf['Reads'].value_counts()

0    1038182
1      47518
Name: Reads, dtype: int64

In [ ]:
# meltdf_clean = meltdf[meltdf['Reads']!=0]
# meltdf_clean.shape

In [17]:
gr_individual = meltdf.groupby(["familyID","anonymized_name","role","Taxonomy","sample_type"]).mean()
# gr_individual = meltdf.groupby(["familyID","anonymized_name","role","Taxonomy","sample_type"])['Reads'].sum()
#  data.groupby(['col1', 'col2'])['col3'].mean()


In [18]:
gr_individual

dna_extracted  \
familyID    anonymized_name role    Taxonomy                                           sample_type                  
family#1020 FS_151          Single  k__Bacteria; p__Acidobacteria; c__Acidobacteria... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__Acidobacteria... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__Acidobacteria... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__Acidobacterii... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__Solibacteres;... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__[Chloracidoba... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__[Chloracidoba... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__[Chloracidoba... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Acidobacteria; c__[Chloracidoba... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
                                    k__Bacteria; p__Actinobacteria; c__Acidimicrobi... saliva                True   
                                                                                       skin                  True   
                                                                                       stool                 True   
...                                                                                                           ...   
family#78   FS_172          Partner k__Bacteria; p__Tenericutes; c__Mollicutes; o__... saliva                True   
                                                                                       skin                  True   
     

In [19]:
gr_individual['Reads'].value_counts()

0.000000    601992
1.000000     16670
0.333333     11565
0.250000      4165
0.666667      3077
0.500000      2051
0.750000       526
Name: Reads, dtype: int64

In [23]:
clean_gr = gr_individual.iloc[:,-1]
clean_gr = clean_gr.reset_index()
clean_gr
# mergedf.iloc[:, -1035:-1]

,familyID,anonymized_name,role,Taxonomy,sample_type,Reads
0,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,saliva,0.000000
1,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,skin,0.000000
2,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,stool,0.000000
3,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,saliva,0.000000
4,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,skin,0.000000
5,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,stool,0.000000
6,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,saliva,0.000000
7,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,skin,0.000000
8,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,stool,0.000000
9,family#1020,FS_151,Single,k__Bacteria; p__Acidobacteria; c__Acidobacteri...,saliva,0.000000


# Filtering Train and Test Datasets

In [ ]:
#FILTER 1
have_dog = "only_non_dog_owners"

#FILTER 2
sample_type = "skin"
# sample_type = "all"
# #FILTER 3
# # not_sample_type = "stool"

# # #FILTER 4
human_role = "Partner"
# human_role = "all"
# #FILTER 5
# not_human_role = "Offspring"

#FILTER 6
# not_familyID = 63

# SPLITTING TRAIN and TEST (test = dogs, train = humans)

In [ ]:
human_data_train = mergedf[mergedf.host_common_name =="human"]

dog_data_test = mergedf[mergedf.host_common_name =="dog"]
dog_data_test.head(1)

In [ ]:
# #testing dog forehead to human right palm hypothesis

# human_data_train = human_data_train[human_data_train.body_site =="UBERON:skin of hand"]
# print(human_data_train.shape)
# dog_data_test = dog_data_test[dog_data_test.body_site =="UBERON:face"]
# print(dog_data_test.shape)

In [ ]:
# Applying filters above
try:
    if have_dog == "only_dog_owners":
        human_data_train = human_data_train[human_data_train.have_dog =="yes"]
    
    elif have_dog == "only_non_dog_owners":
        human_data_train = human_data_train[human_data_train.have_dog =="no"]
    elif have_dog == "all":
        human_data_train = human_data_train
    else:
        human_data_train = human_data_train
except:
    print("no filter here 1")
print(human_data_train.shape)

try:    
    if sample_type!= "all":
        human_data_train = human_data_train[human_data_train.sample_type ==sample_type]
        dog_data_test = dog_data_test[dog_data_test.sample_type ==sample_type]
    elif sample_type == "all":
        human_data_train = human_data_train
        dog_data_test = dog_data_test
    else:
        human_data_train = human_data_train
        dog_data_test = dog_data_test
except:
    print("no filter here 2")
print(human_data_train.shape)  

# try:    
#     if not_sample_type:
#         human_data_train = human_data_train[human_data_train.sample_type !=not_sample_type]
#         dog_data_test = dog_data_test[dog_data_test.sample_type !=not_sample_type]
#     elif sample_type == "all":
#         human_data_train = human_data_train
#         dog_data_test = dog_data_test
# except:
#     print("no filter here 3")
    
try:    
    if human_role != "all":
        human_data_train = human_data_train[human_data_train.role ==human_role]
    elif human_role == "all":
        human_data_train = human_data_train
    else:
        human_data_train = human_data_train
except:
    print("no filter here 4")
print(human_data_train.shape)    
# try:
#     if not_human_role:
#         human_data_train = human_data_train[human_data_train.role !=not_human_role]    
# except:
#     print("no filter here 5")
    
# try:    
#     if not_familyID:
#         human_data_train = human_data_train[human_data_train.role !=not_familyID]
#         dog_data_test = dog_data_test[dog_data_test.role !=not_familyID]  
# except:
#     print("no filter here 6")

# Train/test split

In [ ]:
X_train = human_data_train.iloc[:, -1035:-1]
y_train = human_data_train["familyID"].values.reshape(-1, 1)

y_test = dog_data_test["familyID"].values.reshape(-1, 1)
X_test = dog_data_test.iloc[:, -1035:-1]

# Hyperparameter Selection

In [ ]:
n_estimators = 1000

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=n_estimators)

# Non-Scaled Model Fitting

In [ ]:
#from non-scaled data
rf = rf.fit(X_train, y_train.ravel())
rf.score(X_test, y_test) #geni impurity coefficient

print(f'Test Score: {rf.score(X_test, y_test)}')
print(f'Train Score: {rf.score(X_train, y_train)}')

# Scaled X Model Fitting

In [ ]:
# from sklearn.preprocessing import StandardScaler
# X_scaler = StandardScaler().fit(X_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# from scaled data
# rf_scaled = RandomForestClassifier(n_estimators=n_estimators)
# rf_scaled = rf_scaled.fit(X_train_scaled, y_train.ravel())
# rf_scaled.score(X_test_scaled, y_test) #geni impurity coefficient

# print(f'Test Score: {rf_scaled.score(X_test_scaled, y_test)}')
# print(f'Train Score: {rf_scaled.score(X_train_scaled, y_train)}')

# Evaluate most-guessed family for each dog (7 collection site/predictions per animal) ==> 1 prediction

In [ ]:
y_pred = rf.predict(X_test)

outputdf = pd.DataFrame({"SampleName": dog_data_test["sample_name"],"Anonymized_Name":dog_data_test["anonymized_name"],"Prediction": y_pred, "Actual": y_test.ravel()}).reset_index(drop=True)
print(outputdf.shape)
outputdf.head()

In [ ]:
sumOutput = outputdf.groupby(["Anonymized_Name","Actual","Prediction"]).count()
sumOutput.reset_index()
sumOutput.head()

In [ ]:
# sumOutput.groupby(["Anonymized_Name"])['SampleName'].max()
idx = sumOutput.groupby(["Anonymized_Name"])['SampleName'].transform(max) == sumOutput['SampleName']
maxOutput = sumOutput[idx]
maxOutput = maxOutput.reset_index()
print(maxOutput.shape)
# maxOutput

In [ ]:
maxOutput_noDup = maxOutput.drop_duplicates(subset=['Anonymized_Name'], keep="first")
print(maxOutput_noDup.shape)
# maxOutput_noDup

In [ ]:
maxOutput_C = maxOutput_noDup[maxOutput_noDup.Actual == maxOutput_noDup.Prediction]

print(maxOutput_C.shape)
# maxOutput_C

In [ ]:
maxOutput_I = maxOutput_noDup[maxOutput_noDup.Actual != maxOutput_noDup.Prediction]

print(maxOutput_I.shape)
# maxOutput_I

In [ ]:
correct_total = maxOutput_C["Anonymized_Name"].count()
incorrect_total = maxOutput_I["Anonymized_Name"].count()

accuracy_maxed_pred = correct_total/(correct_total + incorrect_total)
print(f' Accuracy of most-predicted (dog) family: {round(accuracy_maxed_pred,3)}')